# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import math
import xlsxwriter
import requests
from scipy import stats

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'companyName': 'Apple Inc', 'marketcap': 2631414069534, 'week52high': 184.2, 'week52low': 118.52, 'week52highSplitAdjustOnly': 189.84, 'week52lowSplitAdjustOnly': 121.07, 'week52change': 0.2503419581952875, 'sharesOutstanding': 16592210440, 'float': 0, 'avg10Volume': 105079810, 'avg30Volume': 95656063, 'day200MovingAvg': 164.46, 'day50MovingAvg': 169.42, 'employees': 149622, 'ttmEPS': 6.11, 'ttmDividendRate': 0.9066080666503565, 'dividendYield': 0.005935500811309638, 'nextDividendDate': '', 'exDividendDate': '2022-02-02', 'nextEarningsDate': '2022-04-15', 'peRatio': 25.646790815822452, 'beta': 1.2941705342101113, 'maxChangePercent': 60.15029233056528, 'year5ChangePercent': 3.839970311996836, 'year2ChangePercent': 1.285937986056738, 'year1ChangePercent': 0.2698323260214653, 'ytdChangePercent': -0.12747269508759676, 'month6ChangePercent': 0.04371731070358488, 'month3ChangePercent': -0.1376855761591527, 'month1ChangePercent': -0.10343897665411596, 'day30ChangePercent': -0.082544235550298

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.2698323260214653

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    print(data.keys())
    #print(data['AAPL']['stats'])
    for symbol in symbol_string.split(','):
        try:
            final_dataframe = final_dataframe.append(
            pd.Series(
                [
                        symbol,
                        data[symbol]["price"],
                        data[symbol]['stats']["year1ChangePercent"],
                        'N/A'

                ],
                index=my_columns),
                ignore_index = True
            )
        except:
            pass
final_dataframe

dict_keys(['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABMD', 'ABT', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEP', 'AES', 'AFL', 'AIG', 'AIV', 'AIZ', 'AJG', 'AKAM', 'ALB', 'ALGN', 'ALK', 'ALL', 'ALLE', 'ALXN', 'AMAT', 'AMCR', 'AMD', 'AME', 'AMGN', 'AMP', 'AMT', 'AMZN', 'ANET', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'APD', 'APH', 'APTV', 'ARE', 'ATO', 'ATVI', 'AVB', 'AVGO', 'AVY', 'AWK', 'AXP', 'AZO', 'BA', 'BAC', 'BAX', 'BBY', 'BDX', 'BEN', 'BF.B', 'BIIB', 'BIO', 'BK', 'BKNG', 'BKR', 'BLK', 'BLL', 'BMY', 'BR', 'BRK.B', 'BSX', 'BWA', 'BXP', 'C', 'CAG', 'CAH', 'CARR', 'CAT', 'CB', 'CBOE', 'CBRE', 'CCI', 'CCL', 'CDNS', 'CDW', 'CE', 'CERN', 'CF', 'CFG', 'CHD', 'CHRW', 'CHTR', 'CI', 'CINF', 'CL', 'CLX', 'CMA', 'CMCSA'])
dict_keys(['CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP', 'COF', 'COG', 'COO', 'COP', 'COST', 'COTY', 'CPB', 'CPRT', 'CRM', 'CSCO', 'CSX', 'CTAS', 'CTL', 'CTSH', 'CTVA', 'CTXS', 'CVS', 'CVX', 'CXO', 'D', 'DAL', 'DD', 'DE', 'DFS', 'DG', 'DGX', 'DHI', 'DHR', 'DIS', 'DISCA'

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,135.070,0.058319,N/A
1,AAL,16.674,-0.393767,N/A
2,AAP,209.340,0.142951,N/A
3,AAPL,161.659,0.268467,N/A
4,ABBV,161.870,0.495108,N/A
...,...,...,...,...
497,YUM,122.090,0.082149,N/A
498,ZBH,122.900,-0.259201,N/A
499,ZBRA,399.370,-0.194187,N/A
500,ZION,70.210,0.161483,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataframe.sort_values('One-Year Price Return',ascending=False,inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,274,LB,81.900,2.360092,N/A
1,148,DVN,53.009,1.304918,N/A
2,343,NUE,135.280,0.993888,N/A
3,298,MCHP,76.700,0.97841,N/A
4,89,CF,90.330,0.938412,N/A
5,356,OXY,54.145,0.893837,N/A
6,316,MRO,22.568,0.863677,N/A
7,344,NVDA,244.350,0.751949,N/A
8,42,APA,37.052,0.744266,N/A
9,313,MOS,57.140,0.725482,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the size of your portfolio :")
    try:
        protfolio_size = float(portfolio_size)
    except ValueError:
        print("That is not a number \n Please try again :")
        portfolio_size = input("Enter the size of your portfolio :")


In [9]:
portfolio_input()
portfolio_size = float(portfolio_size)

Enter the size of your portfolio :10000000


In [10]:
portfolio_size
#type(len(final_dataframe.index))

10000000.0

In [11]:
position_size = portfolio_size/len(final_dataframe.index)
for i in range(0,len(final_dataframe)):
    final_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i,'Price'])
final_dataframe

/Users/chayanrellan/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,274,LB,81.900,2.360092,2442
1,148,DVN,53.009,1.304918,3772
2,343,NUE,135.280,0.993888,1478
3,298,MCHP,76.700,0.97841,2607
4,89,CF,90.330,0.938412,2214
5,356,OXY,54.145,0.893837,3693
6,316,MRO,22.568,0.863677,8862
7,344,NVDA,244.350,0.751949,818
8,42,APA,37.052,0.744266,5397
9,313,MOS,57.140,0.725482,3500


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [12]:
data.keys()

dict_keys(['YUM', 'ZBH', 'ZBRA', 'ZION', 'ZTS'])

In [13]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
#         print(symbol)
#         print(data[symbol])
#         print(" ")
#         print("------------------------------------------------")
        try:
            hqm_dataframe = hqm_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]["price"],
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A',
                'N/A'

            ],index = hqm_columns
            ),
                ignore_index=True
            )
        except:
            pass


In [14]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,139.530,N/A,0.057187,N/A,-0.250845,N/A,-0.156753,N/A,-0.042486,N/A,N/A
1,AAL,16.117,N/A,-0.385072,N/A,-0.205778,N/A,-0.085384,N/A,-0.178503,N/A,N/A
2,AAP,213.000,N/A,0.146149,N/A,-0.000416,N/A,-0.143579,N/A,-0.08639,N/A,N/A
3,AAPL,157.422,N/A,0.264476,N/A,0.044159,N/A,-0.136855,N/A,-0.103359,N/A,N/A
4,ABBV,160.510,N/A,0.490551,N/A,0.490401,N/A,0.219604,N/A,0.079283,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,123.450,N/A,0.08003,N/A,-0.104559,N/A,-0.13068,N/A,-0.066046,N/A,N/A
498,ZBH,121.850,N/A,-0.268775,N/A,-0.184992,N/A,-0.012486,N/A,-0.02457,N/A,N/A
499,ZBRA,407.950,N/A,-0.195786,N/A,-0.335528,N/A,-0.36109,N/A,-0.114287,N/A,N/A
500,ZION,70.630,N/A,0.162517,N/A,0.143419,N/A,0.047136,N/A,-0.121048,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [15]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]
hqm_dataframe.fillna(0,inplace=True)
for row in hqm_dataframe.index:
    for time_period in time_periods:
        if hqm_dataframe.loc[row, f'{time_period} Return Percentile'] == None:
            hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = 0.0
        hqm_dataframe.loc[row,f'{time_period} Return Percentile'] = stats.percentileofscore(
            hqm_dataframe[f'{time_period} Price Return'],
            hqm_dataframe.loc[row,f'{time_period} Price Return']
        )/100
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,139.530,N/A,0.057187,0.450199,-0.250845,0.073705,-0.156753,0.237052,-0.042486,0.47012,N/A
1,AAL,16.117,N/A,-0.385072,0.025896,-0.205778,0.133466,-0.085384,0.400398,-0.178503,0.055777,N/A
2,AAP,213.000,N/A,0.146149,0.629482,-0.000416,0.533865,-0.143579,0.262948,-0.086390,0.272908,N/A
3,AAPL,157.422,N/A,0.264476,0.802789,0.044159,0.623506,-0.136855,0.280876,-0.103359,0.213147,N/A
4,ABBV,160.510,N/A,0.490551,0.948207,0.490401,0.97012,0.219604,0.934263,0.079283,0.89243,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,123.450,N/A,0.080030,0.49004,-0.104559,0.320717,-0.130680,0.296813,-0.066046,0.36255,N/A
498,ZBH,121.850,N/A,-0.268775,0.081673,-0.184992,0.169323,-0.012486,0.579681,-0.024570,0.535857,N/A
499,ZBRA,407.950,N/A,-0.195786,0.131474,-0.335528,0.02988,-0.361090,0.007968,-0.114287,0.173307,N/A
500,ZION,70.630,N/A,0.162517,0.653386,0.143419,0.804781,0.047136,0.729084,-0.121048,0.159363,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [16]:
from statistics import mean
for row in hqm_dataframe.index:
    momentum_percentiles = []
    
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row,f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row,'HQM Score'] = mean(momentum_percentiles)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,139.530,N/A,0.057187,0.450199,-0.250845,0.073705,-0.156753,0.237052,-0.042486,0.47012,0.307769
1,AAL,16.117,N/A,-0.385072,0.025896,-0.205778,0.133466,-0.085384,0.400398,-0.178503,0.055777,0.153884
2,AAP,213.000,N/A,0.146149,0.629482,-0.000416,0.533865,-0.143579,0.262948,-0.086390,0.272908,0.424801
3,AAPL,157.422,N/A,0.264476,0.802789,0.044159,0.623506,-0.136855,0.280876,-0.103359,0.213147,0.48008
4,ABBV,160.510,N/A,0.490551,0.948207,0.490401,0.97012,0.219604,0.934263,0.079283,0.89243,0.936255
...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,123.450,N/A,0.080030,0.49004,-0.104559,0.320717,-0.130680,0.296813,-0.066046,0.36255,0.36753
498,ZBH,121.850,N/A,-0.268775,0.081673,-0.184992,0.169323,-0.012486,0.579681,-0.024570,0.535857,0.341633
499,ZBRA,407.950,N/A,-0.195786,0.131474,-0.335528,0.02988,-0.361090,0.007968,-0.114287,0.173307,0.085657
500,ZION,70.630,N/A,0.162517,0.653386,0.143419,0.804781,0.047136,0.729084,-0.121048,0.159363,0.586653


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [17]:
hqm_dataframe.sort_values('HQM Score',ascending=False,inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace=True,drop=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,OXY,55.386,N/A,0.901683,0.99004,0.970309,0.998008,0.932990,1.0,0.378736,1.0,0.997012
1,CF,92.001,N/A,0.944926,0.992032,1.007212,1.0,0.522269,0.994024,0.317907,0.998008,0.996016
2,MOS,57.050,N/A,0.736705,0.984064,0.771925,0.988048,0.622065,0.998008,0.291916,0.996016,0.991534
3,APA,37.430,N/A,0.730463,0.982072,0.794751,0.992032,0.479519,0.99004,0.144137,0.966135,0.98257
4,CVX,164.710,N/A,0.520742,0.954183,0.651366,0.98008,0.402694,0.984064,0.186758,0.982072,0.9751
5,BKR,34.700,N/A,0.503381,0.952191,0.424788,0.968127,0.489646,0.992032,0.212301,0.986056,0.974602
6,KR,57.414,N/A,0.600280,0.972112,0.391729,0.958167,0.223346,0.944223,0.212578,0.988048,0.965637
7,LB,79.940,N/A,2.306352,1.0,0.840855,0.996016,0.222023,0.940239,0.081473,0.900398,0.959163
8,NUE,140.994,N/A,1.004256,0.996016,0.249827,0.906375,0.263163,0.960159,0.118439,0.952191,0.953685
9,HAL,35.010,N/A,0.485405,0.944223,0.702186,0.982072,0.580650,0.996016,0.077194,0.888446,0.952689


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [18]:
portfolio_input()

Enter the size of your portfolio :10000000


In [19]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i,'Price'])
hqm_dataframe

/Users/chayanrellan/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,OXY,55.386,3611,0.901683,0.99004,0.970309,0.998008,0.932990,1.0,0.378736,1.0,0.997012
1,CF,92.001,2173,0.944926,0.992032,1.007212,1.0,0.522269,0.994024,0.317907,0.998008,0.996016
2,MOS,57.050,3505,0.736705,0.984064,0.771925,0.988048,0.622065,0.998008,0.291916,0.996016,0.991534
3,APA,37.430,5343,0.730463,0.982072,0.794751,0.992032,0.479519,0.99004,0.144137,0.966135,0.98257
4,CVX,164.710,1214,0.520742,0.954183,0.651366,0.98008,0.402694,0.984064,0.186758,0.982072,0.9751
5,BKR,34.700,5763,0.503381,0.952191,0.424788,0.968127,0.489646,0.992032,0.212301,0.986056,0.974602
6,KR,57.414,3483,0.600280,0.972112,0.391729,0.958167,0.223346,0.944223,0.212578,0.988048,0.965637
7,LB,79.940,2501,2.306352,1.0,0.840855,0.996016,0.222023,0.940239,0.081473,0.900398,0.959163
8,NUE,140.994,1418,1.004256,0.996016,0.249827,0.906375,0.263163,0.960159,0.118439,0.952191,0.953685
9,HAL,35.010,5712,0.485405,0.944223,0.702186,0.982072,0.580650,0.996016,0.077194,0.888446,0.952689


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [20]:
writer = pd.ExcelWriter('momentum_strategy.xlsx',engine='xlsxwriter')

hqm_dataframe.to_excel(writer,sheet_name='Momentum Strategy',index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [21]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [22]:
column_formats = {
    'A':['Ticker',string_template],
    'B':['Price',dollar_template],
    'C':['Number of Shares to Buy',integer_template],
    'D':['One-Year Price Return',percent_template],
    'E':['One-Year Return Percentile',percent_template],
    'F':['Six-Month Price Return',percent_template],
    'G':['Six-Month Return Percentile',percent_template],
    'H':['Three-Month Price Return',percent_template],
    'I':['Three-Month Return Percentile',percent_template],
    'J':['One-Month Price Return',percent_template],
    'K':['One-Month Return Percentile',percent_template],
    'L':['HQM Score',percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}',25,column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1',column_formats[column][0],column_formats[column][1])

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [23]:
writer.save()